### LLM для задач генерации
#### decoder-only (causal lm)
универсальный вариант для задач генерации

 **инструктивные модели**
1.   IlyaGusev/saiga_nemo_12b / IlyaGusev/saiga_gemma3_12b / IlyaGusev/saiga_llama3_8b - лучший вариант для русского
2. mistralai/Mistral-7B-Instruct-v0.2 - базовый варинат на английском

**НЕинструктивные модели**
1.   ai-forever/rugpt3small_based_on_gpt2 (small, medium, large) - маленькие по памяти варианты не Instruct
2. ai-forever/ruGPT-3.5-13B - самый жирный вариант не Instruct модели
3. mistralai/Mistral-7B-v0.1 - базовый варинат на английском

#### encoder-decoder (seq2seq)
суммаризация, перевод, перефразирование - зачастую меньше чем decoder-only модели, но могут справляться лучше при файнтюнинге

1. cointegrated/rut5-base-multitask - маленькая multitask моделька, также работает с Instruct токенами
2. google/flan-t5-xl (3b) - мультиязыная моделька (с поддержкой русского), без instruct токенов

**файнтюны под отдельные таски**

3. IlyaGusev/rut5_base_sum_gazeta / cointegrated/rut5-base-absum - быстрый вариант чисто для суммаризации
4. Helsinki-NLP/opus-mt-ru-en - быстрый сота перевод
5. cointegrated/rut5-base-paraphraser - единственный вариант для перефразирования

#### encoder-only
подходит только для задач завязанных на эмбедднгах

1.   Новый пункт
2.   Новый пункт



### Метрики для задач генерации

1.   **BLEU** - сравнивает n-граммы (последовательности слов) сгенерированного текста с эталонным (быстрая, но не учитывает смысл)
2.   **ROUGE** - ориентирована на recall (полноту) — насколько эталонные n-граммы присутствуют в сгенерированном тексте
3.   **METEOR** - улучшение BLEU, учитывает синонимы, стемминг и согласование с эталоном
4.   **BERTScore** - использует контекстуальные эмбеддинги BERT для сравнения смысловой схожести, а не просто совпадения слов (универсальная метрика для любых задач генерации, где важен смысл, современный стандарт)
5.   **Perplexity** - измеряет, насколько модель большая суммарная вероятность и beamов в сгенерированном тексте, не сравнивает с эталоном

In [ ]:
!pip install -q torch transformers datasets optuna peft bitsandbytes
!pip install -q evaluate rouge_score bert_score
# !pip install -q -U torch torchvision torchaudio
# !pip install -q -U transformers accelerate bitsandbytes
# !pip install -q -U peft
# !pip install -U peft transformers accelerate bitsandbytes
# !pip install -q optuna evaluate rouge_score bert_score

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_SILENT"] = "true"

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    pipeline,
    GenerationConfig,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    get_linear_schedule_with_warmup,
    get_constant_schedule_with_warmup,
    TrainingArguments,
    Trainer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from datasets import load_dataset
from torch.nn.utils import clip_grad_norm_

import evaluate
import optuna
import numpy as np
import pandas as pd

import gc
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    PeftModel,
    PeftConfig
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(f"{torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

cuda
NVIDIA L4
23.80 GB


In [ ]:
dataset = load_dataset("IlyaGusev/gazeta")
train_df = pd.DataFrame(dataset['train'].select(range(2000)))
val_df = pd.DataFrame(dataset['validation'].select(range(250)))
print(train_df.shape)
print(val_df.shape)
train_df.head()

(2000, 5)
(250, 5)


,text,summary,title,date,url
0,Сегодня транспортный налог начисляется в завис...,С 2011 года правительство отменяет самый раздр...,Налог в бак,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...
1,Словосочетание «музыкальный кинофестиваль» уже...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...
2,После более чем 12-часовых консультаций Совет ...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...
3,"Жертвами урагана «Агата», обрушившегося на Цен...",Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...
4,Решение ограничить рост тарифов естественных м...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...


### seq2seq lm inference

In [ ]:
model_name = "cointegrated/rut5-base-multitask"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)
print(config)

T5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "gradient_checkpointing": false,
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.57.1",
  "use_cache": true,
  "vocab_size": 30000
}



In [ ]:
print(tokenizer.pad_token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

<pad>


In [ ]:
input = tokenizer('headline |' + val_df.loc[1].summary, return_tensors='pt').to(device)
with torch.no_grad():
    target = model.generate(**input, num_beams=5)
target = tokenizer.decode(target.squeeze(0), skip_special_tokens=True)
print(target)

Трамп призвала лидера КНДР не проводить испытания нового стратегического


In [ ]:
model.eval()
all_preds = []
all_targets = val_df['title'].to_list()

for input_text in tqdm(val_df['summary']):
    input = 'headline |' + input_text
    input = tokenizer(input, return_tensors='pt').to(device)
    with torch.no_grad():
        target = model.generate(**input, num_beams=5)
    target = tokenizer.decode(target.squeeze(0), skip_special_tokens=True)
    all_preds.append(target)

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

rouge_results = rouge.compute(predictions=all_preds, references=all_targets, use_stemmer=True)
bertscore_results = bertscore.compute(predictions=all_preds, references=all_targets, lang="ru")

for res in rouge_results:
    print(f'{res}: {rouge_results[res]}')
for res in list(bertscore_results.keys())[:-1]:
    print(f'bertscore {res}: {np.mean(bertscore_results[res]).item()}')

rouge1: 0.053566666666666665
rouge2: 0.0013333333333333333
rougeL: 0.05326666666666667
rougeLsum: 0.05333333333333334
bertscore precision: 0.7480756293535232
bertscore recall: 0.6904822115898133
bertscore f1: 0.7176241570711136


In [ ]:
idx = 0
print(all_targets[idx])
print()
print(all_preds[idx])

Дорогой 2020-й: какие продукты подскочат в цене

Стоимость молочных продуктов в 2020 году может вырасти гораздо выше инфляции


In [ ]:
class TitlePredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=None, prompt_f='headline | ', prompt_b='', train=True):
        self.train = train
        self.inputs = []
        self.targets = []
        for idx, row in df.iterrows():
            self.inputs.append(prompt_f + row['summary'] + prompt_b)
            if self.train:
              self.targets.append(row['title'])
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input = self.tokenizer(
            self.inputs[idx],
            max_length=self.max_len, # максимльная длина в токенах или None если ограничиваем архитектурой модели
            truncation=True, # обрезаем до max_length или макс входа модели
            padding=False, # "max_length" паддинг, у нас динамический
            return_tensors="pt" # возвращаем тензоры
        )
        if self.train:
            target = self.tokenizer(
                self.targets[idx],
                max_length=self.max_len,
                truncation=True,
                padding=False,
                return_tensors="pt"
            )

            return {
                'input_ids': input['input_ids'].squeeze(0),
                'attention_mask': input['attention_mask'].squeeze(0),
                'labels': target['input_ids'].squeeze(0)
            }
        return {
                'input_ids': input['input_ids'].squeeze(0),
                'attention_mask': input['attention_mask'].squeeze(0)
            }

In [ ]:
collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, # обязательно tokenizer
    padding='longest', # паддинг до самого длиного элемента в батче
    max_length=1024,
    # pad_to_multiple_of=8, # оптимизация для gpu
)

# def custom_collate_fn(batch):
#     # собираем все input_ids, attention_mask, labels
#     input_ids = [item['input_ids'] for item in batch]
#     attention_mask = [item['attention_mask'] for item in batch]
#     labels = [item['labels'] for item in batch]

#     # находим максимальные длины
#     max_input_len = max(len(seq) for seq in input_ids)
#     max_label_len = max(len(seq) for seq in labels)

#     # округляем до кратного 8 для эффективности
#     # max_input_len = (max_input_len + 7) // 8 * 8
#     # max_label_len = (max_label_len + 7) // 8 * 8

#     batch_size = len(batch)

#     # паддинг для input_ids
#     padded_input_ids = torch.full((batch_size, max_input_len), tokenizer.pad_token_id, dtype=torch.long)
#     for i, seq in enumerate(input_ids):
#         length = min(len(seq), max_input_len)
#         padded_input_ids[i, :length] = seq[:length]

#     # Паддинг для attention_mask
#     padded_attention_mask = torch.zeros((batch_size, max_input_len), dtype=torch.long)
#     for i, seq in enumerate(attention_mask):
#         length = min(len(seq), max_input_len)
#         padded_attention_mask[i, :length] = 1

#     # паддинг для labels (с заменой pad_token_id на -100)
#     padded_labels = torch.full((batch_size, max_label_len), -100, dtype=torch.long)
#     for i, seq in enumerate(labels):
#         length = min(len(seq), max_label_len)
#         padded_labels[i, :length] = seq[:length]

#     return {
#         'input_ids': padded_input_ids,
#         'attention_mask': padded_attention_mask,
#         'labels': padded_labels
#     }

In [ ]:
train_dataset = TitlePredictionDataset(train_df, tokenizer, train=True)
val_dataset = TitlePredictionDataset(val_df, tokenizer, train=True)
val_dataset_inf = TitlePredictionDataset(val_df, tokenizer, train=False)
batch_size=8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator) # custom_collate_fn
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator) # custom_collate_fn
val_loader_inf = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator) # custom_collate_fn

In [ ]:
model.eval()
all_preds = []
all_targets = []

for batch in tqdm(val_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels']

    with torch.no_grad():
        preds = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=512, # максимальная длина output в токенах
            num_beams=5, # колво beamов, 1 - жадный, 4-5 - хорошее качество, 10+ - лучшее качество но медленно
            early_stopping=True, # остановка когда на всех beamах <eos>
            do_sample=True, # False=детерминированный поиск, True=стохастический
            temperature=0.7, # температура (1 - оригинальное распределение, чем больше тем креативнее)
            top_p=0.95, # nucleus sampling, 0.9-0.95 стандарт (только с do_sample=True)
            # top_k=50, # top-k sampling (альтернатива top_p)
            repetition_penalty=1.2, # 1.0 - нет, 1.2 - стандарт, 2.0 - сильный штраф
            # no_repeat_ngram_size=2, # запрет повтора n-грамм
            length_penalty=0.7, # <1 короче, =1 нейтрально, >1 длиннее (для beam search)
            # min_length=10, # минимальная длина
            # num_return_sequences=1, # сколько вариантов вернуть (для beam или sampling)
            # pad_token_id=tokenizer.pad_token_id,
            # eos_token_id=tokenizer.eos_token_id
        )

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # для таргетов заменяем -100 (подставляет коллатор тк -100 не учитывается при подсчете torch метрик
    # => модель не учится предсказывать pad токены) на pad токен
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    all_preds.extend(decoded_preds)
    all_targets.extend(decoded_labels)

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

def evaluate_model(model, dataloader, tokenizer, generation_config):
    model.eval()
    all_preds = []
    all_targets = []

    for batch in tqdm(dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        with torch.no_grad():
            preds = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                generation_config=generation_config
            )

        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        # для таргетов заменяем -100 (подставляет коллатор тк -100 не учитывается при подсчете torch метрик
        # => модель не учится предсказывать pad токены) на pad токен
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        all_preds.extend(decoded_preds)
        all_targets.extend(decoded_labels)

    rouge_results = rouge.compute(predictions=all_preds, references=all_targets, use_stemmer=True)
    bertscore_results = bertscore.compute(predictions=all_preds, references=all_targets, lang="ru")
    return rouge_results, bertscore_results

In [ ]:
def objective(trial):
    # Подбор гиперпараметров генерации
    max_new_tokens = trial.suggest_int('max_new_tokens', 8, 128)
    num_beams = trial.suggest_int('num_beams', 5, 8)
    temperature = trial.suggest_float('temperature', 0.3, 1.5)
    repetition_penalty = trial.suggest_float('repetition_penalty', 1.0, 2.0)
    length_penalty = trial.suggest_float('length_penalty', 1.0, 2.0)

    val_dataset = TitlePredictionDataset(val_df, tokenizer) #, prompt_b=prompt_b)
    val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collator)

    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        num_beams=num_beams,
        early_stopping=True,
        do_sample=True,
        temperature=temperature,
        repetition_penalty=repetition_penalty,
        length_penalty=length_penalty
    )

    rouge_score, bert_score = evaluate_model(model, val_loader, tokenizer, generation_config)
    return np.mean(bert_score['f1']).item()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: {trial.params}")

[I 2025-11-17 12:25:01,779] A new study created in memory with name: no-name-f4d418a4-de72-46b8-9605-df4839ba1742


  0%|          | 0/75 [00:00<?, ?it/s]

`generation_config` default values have been modified to match model-specific defaults: {'pad_token_id': 0, 'eos_token_id': 1, 'decoder_start_token_id': 0}. If this is not desired, please set these values explicitly.
[I 2025-11-17 12:25:53,394] Trial 0 finished with value: 0.7203515166044235 and parameters: {'max_new_tokens': 120, 'num_beams': 8, 'temperature': 0.30230293212978965, 'repetition_penalty': 1.3992205983628088, 'length_penalty': 1.6094598337145607}. Best is trial 0 with value: 0.7203515166044235.


Best trial:
  Value: 0.7203515166044235
  Params: {'max_new_tokens': 120, 'num_beams': 8, 'temperature': 0.30230293212978965, 'repetition_penalty': 1.3992205983628088, 'length_penalty': 1.6094598337145607}


### seq2seq lm finetuning

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,  # для seq2seq моделей
    inference_mode=False,
    r=8,  # rank матриц LoRA
    lora_alpha=32, # множитель масштабирования, обычно r * 4
    lora_dropout=0.1, # дропаут для LoRA слоев
    target_modules=["q", "v", "k", "o", "wi", "wo"],  # модули T5 которые обучаем с LoRA
    # bias="none", # не трогаем bias
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,310,144 || all params: 246,619,392 || trainable%: 0.9367


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

def compute_metrics(preds, targets):
    rouge_results = rouge.compute(
        predictions=preds,
        references=targets,
        use_stemmer=True
    )

    bertscore_results = bertscore.compute(
        predictions=preds,
        references=targets,
        lang="ru"
    )

    return {
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
        'bertscore_f1': np.mean(bertscore_results['f1'])
    }

In [ ]:
def evaluate_model(model, dataloader, tokenizer, generation_config=None):
    model.eval()
    all_preds = []
    all_targets = []

    for batch in tqdm(dataloader, desc="validation"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels']

        with torch.no_grad():
            preds = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                # generation_config=generation_config,
                num_beams=5
            )

        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        all_preds.extend(decoded_preds)
        all_targets.extend(decoded_labels)

    return compute_metrics(all_preds, all_targets), all_preds, all_targets

In [ ]:
# base_generation_config = GenerationConfig(
#     max_new_tokens=1024,
#     num_beams=5,
#     early_stopping=True,
#     do_sample=True,
#     temperature=1.1,
#     repetition_penalty=1.2,
#     length_penalty=0.8
# )

base_generation_config = GenerationConfig(
    # max_new_tokens=1024,
    num_beams=5,
    # early_stopping=True,
    # do_sample=True,
    # temperature=1.1,
    # repetition_penalty=1.2,
    # length_penalty=0.8
)

#### Schedulers
1. get_constant_schedule(optimizer) - константный lr
2. get_constant_schedule_with_warmup(optimizer, num_warmup_steps) - линейное возврастание в течении warmup_steps до initial_lr, после константный initial_lr
3. get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps) - сначала линейное возврастание до initial_lr в течении warmup_steps, потом линейное падение
4. get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps) - сначала линейное возврастание до initial_lr в течении warmup_steps, потом паднение по косинусу (num_cycles = 0.5 - количество волн)


In [ ]:
# параметры обучения
num_epochs = 3
learning_rate = 1e-4
warmup_ratio = 0.1
max_grad_norm = 1.0
save_dir = "./seq2seq_ckpts"
os.makedirs(save_dir, exist_ok=True)

# оптимизатор
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=0.01
)

# scheduler (Linear with Warmup - лучший для LoRA)
total_steps = len(train_loader) * num_epochs
warmup_steps = int(total_steps * warmup_ratio)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# сохранение метрик
best_rougeL = 0
train_losses = []
val_metrics_history = []

for epoch in range(num_epochs):
    # train
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()

        # gradient clipping
        clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
        })

    train_loss = epoch_loss / len(train_loader)
    train_losses.append(train_loss)

    # validation
    val_metrics, val_preds, val_targets = evaluate_model(model, val_loader, tokenizer)
    val_metrics_history.append(val_metrics)

    print(f"   Epoch {epoch+1}/{num_epochs}:")
    print(f"   Train Loss: {train_loss:.4f}")
    print(f"   ROUGE-1: {val_metrics['rouge1']:.4f}")
    print(f"   ROUGE-2: {val_metrics['rouge2']:.4f}")
    print(f"   ROUGE-L: {val_metrics['rougeL']:.4f}")
    print(f"   BERTScore F1: {val_metrics['bertscore_f1']:.4f}")

    # сохраняем чекпоинт
    checkpoint_path = os.path.join(save_dir, f"epoch-{epoch+1}")
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)
    print(f"Checkpoint saved: {checkpoint_path}")

    # cохраняем лучшую модель
    if val_metrics['rougeL'] > best_rougeL:
        best_rougeL = val_metrics['rougeL']
        best_checkpoint_path = os.path.join(save_dir, "best_model")
        model.save_pretrained(best_checkpoint_path)
        tokenizer.save_pretrained(best_checkpoint_path)
        print(f"New best checkpoint with ROUGE-L: {best_rougeL:.4f}")

Epoch 1/3:   0%|          | 0/1250 [00:00<?, ?it/s]

validation:   0%|          | 0/63 [00:00<?, ?it/s]

   Epoch 1/3:
   Train Loss: 3.5932
   ROUGE-1: 0.0449
   ROUGE-2: 0.0027
   ROUGE-L: 0.0455
   BERTScore F1: 0.7157
Checkpoint saved: ./seq2seq_ckpts/epoch-1
New best checkpoint with ROUGE-L: 0.0455


Epoch 2/3:   0%|          | 0/1250 [00:00<?, ?it/s]

validation:   0%|          | 0/63 [00:00<?, ?it/s]

   Epoch 2/3:
   Train Loss: 3.1085
   ROUGE-1: 0.0519
   ROUGE-2: 0.0047
   ROUGE-L: 0.0520
   BERTScore F1: 0.7167
Checkpoint saved: ./seq2seq_ckpts/epoch-2
New best checkpoint with ROUGE-L: 0.0520


Epoch 3/3:   0%|          | 0/1250 [00:00<?, ?it/s]

validation:   0%|          | 0/63 [00:00<?, ?it/s]

   Epoch 3/3:
   Train Loss: 2.8526
   ROUGE-1: 0.0450
   ROUGE-2: 0.0053
   ROUGE-L: 0.0458
   BERTScore F1: 0.7158
Checkpoint saved: ./seq2seq_ckpts/epoch-3


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(model, "./seq2seq_ckpts/best_model")
tokenizer = AutoTokenizer.from_pretrained("./seq2seq_ckpts/best_model")

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./seq2seq_ckpts_v1",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-4,
    weight_decay=0.01,

    # scheduler
    lr_scheduler_type="linear",
    warmup_ratio=0.1,

    # gradient clipping & acc steps
    max_grad_norm=1.0,
    gradient_accumulation_steps=1,

    # сохранение и логирование
    logging_steps=50,
    #logging_dir="./seq2seq_logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="rougeL",
    greater_is_better=True,

    # генерация для валидации
    predict_with_generate=True,
    # generation_max_length=128,
    # generation_config=,
    generation_num_beams=5,

    # Прочие настройки
    fp16=torch.cuda.is_available(),
    report_to=[],
    seed=42
)

def compute_metrics_for_trainer(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return compute_metrics(decoded_preds, decoded_labels)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_for_trainer,
)

trainer.train()
trainer.save_model()
# tokenizer.save_pretrained("./trainer_checkpoints/final_model")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Bertscore F1
1,0.000000,nan,0.078400,0.007000,0.077467,0.713491
2,0.000000,nan,0.078400,0.007000,0.077467,0.713491


KeyboardInterrupt: 

### causal lm inference

In [ ]:
model_name = "Qwen/Qwen2.5-7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# конфигурация 4-битной квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True, # вложенная квантизация
    # bnb_4bit_quant_type="nf4", # тип квантизации (nf4 лучше для обучения)
    bnb_4bit_compute_dtype=torch.float16  # тип для вычислений
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # load_in_4bit=True,
    device_map="auto", # автоматическое распределение по GPU/CPU
    dtype=torch.float16
)
# model.to(device)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(tokenizer.pad_token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

<|endoftext|>


In [ ]:
messages = [
    {"role": "system", "content": "Ты — ассистент для создания заголовков новостей. Ты получаешь текст новости и возвращаешь только заголовок без лишних слов."},
    {"role": "user", "content": val_df.loc[0].summary}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)
print(text)
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

<|im_start|>system
Ты — ассистент для создания заголовков новостей. Ты получаешь текст новости и возвращаешь только заголовок без лишних слов.<|im_end|>
<|im_start|>user
В уходящем году инфляция в России находится на историческом минимуме. В следующем году ожидается, что она также будет минимальной. Однако стоимость ряда продуктов и напитков в 2020 году может вырасти гораздо выше инфляции. Это касается молочных продуктов. Вырастет в цене водка, коньяк и вино, продолжит дорожать гречка.<|im_end|>
<|im_start|>assistant

Цены на некоторые продукты и напитки могут вырасти выше инфляции в следующем году


In [ ]:
class TitlePredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512, train=False):
        self.df = df
        self.tokenizer = tokenizer
        self.system_prompt = "Ты — ассистент для создания заголовков новостей. Ты получаешь текст новости и возвращаешь только заголовок без лишних слов."
        self.base_prompt = ""
        self.max_len = max_len
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input = row['summary']
        target = row['title'] if self.train else ''

        if self.train:
            # для обучения нам нужен полный диалог с ответом ассистента
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.base_prompt + input},
                {"role": "assistant", "content": target}
            ]

            # применяем шаблон для получения отформатированного текста
            full_text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,  # получаем строку, а не токены
                add_generation_prompt=False  # не добавляем промпт генерации, так как он сам добавляется при ответе assistant
            )

            # токенизируем отформатированный текст
            full_text_enc = self.tokenizer(
                full_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            assistant_start = full_text.find("<|im_start|>assistant")
            prompt_text = full_text[:assistant_start]

            prompt_enc = self.tokenizer(
                prompt_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            prompt_len = prompt_enc['input_ids'].shape[1]

            labels = full_text_enc['input_ids'].clone().squeeze(0)
            labels[:prompt_len] = -100

            return {
                'input_ids': full_text_enc['input_ids'].squeeze(0),
                'attention_mask': full_text_enc['attention_mask'].squeeze(0),
                'labels': labels
            }

        else:
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.base_prompt + input}
            ]

            prompt_text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            prompt_enc = self.tokenizer(
                prompt_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt",
            )

            return {
                'input_ids': prompt_enc['input_ids'].squeeze(0),
                'attention_mask': prompt_enc['attention_mask'].squeeze(0)
            }

In [ ]:
class TitlePredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512, train=False):
        self.df = df
        self.tokenizer = tokenizer
        self.system_prompt = "Ты — ассистент для создания заголовков новостей. Ты получаешь текст новости и возвращаешь только заголовок без лишних слов."
        self.max_len = max_len
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input_text = row['summary']
        target_text = row['title'] if self.train else ''

        if self.train:
            # ФОРМИРУЕМ ПРОМПТ БЕЗ ОТВЕТА
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": f"Сгенерируй заголовок для новости: {input_text}"}
            ]

            # Промпт для модели (без ответа)
            prompt = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True  # ДОБАВЛЯЕМ ТОКЕН НАЧАЛА ГЕНЕРАЦИИ
            )

            # ОТВЕТ АССИСТЕНТА (только заголовок)
            assistant_response = target_text

            # ПОЛНЫЙ ТЕКСТ: промпт + ответ
            full_text = prompt + assistant_response

            # Токенизируем полный текст
            encodings = self.tokenizer(
                full_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            # Токенизируем только промпт чтобы найти где начинается ответ
            prompt_enc = self.tokenizer(
                prompt,  # ТОЛЬКО ПРОМПТ
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            prompt_len = prompt_enc['input_ids'].shape[1]

            # МАСКИРУЕМ ВЕСЬ ПРОМПТ, ОСТАВЛЯЕМ ТОЛЬКО ОТВЕТ
            labels = encodings['input_ids'].clone().squeeze(0)
            labels[:prompt_len] = -100

            return {
                'input_ids': encodings['input_ids'].squeeze(0),
                'attention_mask': encodings['attention_mask'].squeeze(0),
                'labels': labels
            }
        else:
            # Для инференса - только промпт
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": f"Сгенерируй заголовок для новости: {input_text}"}
            ]

            prompt = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            encodings = self.tokenizer(
                prompt,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            return {
                'input_ids': encodings['input_ids'].squeeze(0),
                'attention_mask': encodings['attention_mask'].squeeze(0)
            }

In [ ]:
# ПЕРЕСОЗДАЕМ ДАТАСЕТ С ИСПРАВЛЕННЫМ КЛАССОМ
train_dataset = TitlePredictionDataset(train_df, tokenizer, train=True)
val_dataset_wlabels = TitlePredictionDataset(val_df, tokenizer, train=True)

# ПРОВЕРЯЕМ ДАННЫЕ
debug_batch = next(iter(DataLoader(train_dataset, batch_size=2, collate_fn=collator)))
idx = 0

print("=== ИСПРАВЛЕННЫЕ ДАННЫЕ ===")
input_text = tokenizer.decode(debug_batch['input_ids'][idx], skip_special_tokens=False)
labels = debug_batch['labels'][idx]
labels_text = tokenizer.decode(labels[labels != -100], skip_special_tokens=True)

print(f"Input (первые 500 символов): {input_text[:500]}...")
print(f"Labels (только незамаскированные): '{labels_text}'")
print(f"Длина незамаскированных токенов: {(labels != -100).sum().item()}")

# ДОЛЖНО БЫТЬ:
# Input: <system>...<user>... (весь промпт ДО начала ответа)
# Labels: 'Торговля слабо оживает' (ТОЛЬКО заголовок, БЕЗ "assistant" и других токенов)

=== ИСПРАВЛЕННЫЕ ДАННЫЕ ===
Input (первые 500 символов): <|im_start|>system
Ты — ассистент для создания заголовков новостей. Ты получаешь текст новости и возвращаешь только заголовок без лишних слов.<|im_end|>
<|im_start|>user
Сгенерируй заголовок для новости: С 2011 года правительство отменяет самый раздражающий граждан налог – транспортный. Но поборы автомобилистов не прекратятся – налоги завуалируют в бензиновые акцизы и платные дороги, а цены на товары подскочат. Зато теперь собираемые деньги обещают пустить только на строительство и содержание до...
Labels (только незамаскированные): 'Налог в бак'
Длина незамаскированных токенов: 6


In [ ]:
val_dataset_inf = TitlePredictionDataset(val_df, tokenizer, train=False)
val_dataset = TitlePredictionDataset(val_df, tokenizer, train=True)
train_dataset = TitlePredictionDataset(train_df, tokenizer, train=True)

In [ ]:
collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, # обязательно tokenizer
    padding='longest', # паддинг до самого длиного элемента в батче
    max_length=None, # так как уже ограничили в датасете
    # pad_to_multiple_of=8, # оптимизация для gpu
)

In [ ]:
batch_size=4
val_loader_inf = DataLoader(val_dataset_inf, batch_size=batch_size, shuffle=False, collate_fn=collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collator)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collator)

In [ ]:
generation_config = GenerationConfig.from_pretrained(model_name)
generation_config.max_new_tokens = 512
# generation_config.length_penalty = 0.7
print(generation_config)

GenerationConfig {
  "bos_token_id": 151643,
  "do_sample": true,
  "eos_token_id": [
    151645,
    151643
  ],
  "max_new_tokens": 512,
  "pad_token_id": 151643,
  "repetition_penalty": 1.05,
  "temperature": 0.7,
  "top_k": 20,
  "top_p": 0.8
}



In [ ]:
model.eval()
all_preds = []
all_targets = val_df['title'].to_list()

for batch in tqdm(val_loader_inf):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    with torch.no_grad():
        preds = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config,
        )

    for idx, val in enumerate(preds):
        pred = val[len(input_ids[idx]):]
        decoded_pred = tokenizer.decode(pred, skip_special_tokens=True)
        all_preds.append(decoded_pred)

  0%|          | 0/63 [00:00<?, ?it/s]

In [ ]:
idx = 0
print('target:', all_targets[idx])
print('model:', all_preds[idx])

target: Дорогой 2020-й: какие продукты подскочат в цене
model: Цены на продукты и напитки могут вырасти выше инфляции в следующем году


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

rouge_results = rouge.compute(predictions=all_preds, references=all_targets, use_stemmer=True)
bertscore_results = bertscore.compute(predictions=all_preds, references=all_targets, lang="ru")

for res in rouge_results:
    print(f'{res}: {rouge_results[res]}')
for res in list(bertscore_results.keys())[:-1]:
    print(f'bertscore {res}: {np.mean(bertscore_results[res]).item()}')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

rouge1: 0.09486666666666667
rouge2: 0.007333333333333333
rougeL: 0.09559999999999999
rougeLsum: 0.09453333333333333
bertscore precision: 0.6903532433509827
bertscore recall: 0.7199786155223846
bertscore f1: 0.7043537366390228


In [ ]:
batch = next(iter(train_loader))
print(f"Input shape: {batch['input_ids'].shape}")
print(f"Labels shape: {batch['labels'].shape}")
idx = 1
print(batch['input_ids'][idx])
print(batch['attention_mask'][idx])
print(batch['labels'][idx])

input_text = tokenizer.decode(batch['input_ids'][idx], skip_special_tokens=True)
labels = batch['labels'][idx]
labels_text = tokenizer.decode(labels[labels != -100], skip_special_tokens=True)

print(f"Input: {input_text}")
print(f"Labels: {labels_text}")

Input shape: torch.Size([4, 165])
Labels shape: torch.Size([4, 165])
tensor([151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151644,
          8948,    198,  33995,   4552,   1959,  20396, 128698,  34011,  18673,
         19849, 141132,  71490,  72661,  89777,  48951, 135852,     13,  50318,
          4552, 130783, 131260,  70895, 141268,   7587,   5805,  58297,   1478,
        131260,  73626,  71490,  72661,  14746,  91357, 127225, 131722,  91107,
            13, 151645,    198, 151644,    872,    198,  16206,   1456, 138616,
         12228,   7665,   5474,  59388,  15952,   1504,    220,     17,     15,
            16,     15, 128536,  18108,  22787,   2247,  43838, 126421, 132835,
         13695,  17686, 133911,   6715,  95965,  36305, 130811,   1802,   8215,
        130079,  61676, 128693, 136739,  36673,   1478, 128696, 127912,  13103,
         61676,     13,  48420,  83250,  37421, 128148,  13039,   4552,  88615,
         37557,   7587,  20811, 131586,  13132,    

### causal lm LoRA finetuning

In [1]:
!pip -q install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 31.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
from trl import SFTTrainer
from datasets import Dataset

In [3]:
model_name = "Qwen/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [4]:
import pandas as pd
from datasets import load_dataset

In [5]:
dataset = load_dataset("IlyaGusev/gazeta")
train_df = pd.DataFrame(dataset['train'].select(range(2000)))
val_df = pd.DataFrame(dataset['validation'].select(range(250)))
print(train_df.shape)
print(val_df.shape)
train_df.head()

README.md: 0.00B [00:00, ?B/s]

default/train/0000.parquet:   0%|          | 0.00/252M [00:00<?, ?B/s]

default/train/0001.parquet:   0%|          | 0.00/22.7M [00:00<?, ?B/s]

default/validation/0000.parquet:   0%|          | 0.00/27.8M [00:00<?, ?B/s]

default/test/0000.parquet:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

(2000, 5)
(250, 5)


,text,summary,title,date,url
0,Сегодня транспортный налог начисляется в завис...,С 2011 года правительство отменяет самый раздр...,Налог в бак,2010-06-01 10:35:49,https://www.gazeta.ru/auto/2010/05/31_a_337771...
1,Словосочетание «музыкальный кинофестиваль» уже...,"Британские затворники, московские модники, бра...","Секс, наркотики и темный зал",2010-06-01 10:42:59,https://www.gazeta.ru/culture/2010/06/01/a_337...
2,После более чем 12-часовых консультаций Совет ...,Совбез ООН собрался на экстренное совещание дл...,Осудить и отпустить,2010-06-01 11:00:30,https://www.gazeta.ru/politics/2010/06/01_a_33...
3,"Жертвами урагана «Агата», обрушившегося на Цен...",Ураган «Агата» в Центральной Америке унес жизн...,«Агата» открыла страшный сезон,2010-06-01 11:05:30,https://www.gazeta.ru/social/2010/06/01/337799...
4,Решение ограничить рост тарифов естественных м...,Правительство хочет сдержать рост тарифов есте...,Тарифы инфляцию не остановят,2010-06-01 11:48:50,https://www.gazeta.ru/financial/2010/06/01/337...


In [7]:
formatted_data = []
for text, summary in zip(train_df['text'], train_df['summary']):
    formatted_data.append(f"### Text: {text}\n### Summary: {summary}")
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
from transformers import Trainer

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./lora-output",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model()

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/tmp/ipython-input-4076308982.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.


Step,Training Loss
10,1.701400
20,1.846600
30,1.789800
40,1.692300
50,1.759200
60,1.769700
70,1.634000
80,1.633800
90,1.677100
100,1.594600


In [22]:
training_args = TrainingArguments(
    output_dir="./lora-output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args
)

trainer.train()
trainer.save_model()

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'

In [23]:
training_args = TrainingArguments(
    output_dir="./output",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    report_to=[]
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer
)

trainer.train()
trainer.save_model()

TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'dataset_text_field'

### causal lm LoRA finetuning

In [ ]:
!pip -y uninstall bitsandbytes
!pip install bitsandbytes


Usage:   
  pip3 <command> [options]

no such option: -y


In [ ]:
!pip install -q evaluate rouge_score bert_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    get_linear_schedule_with_warmup,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import bitsandbytes as bnb
from tqdm.notebook import tqdm
import pandas as pd
import os
from evaluate import load

In [ ]:
dataset = load_dataset("IlyaGusev/gazeta")
train_df = pd.DataFrame(dataset['train'].select(range(2000)))
val_df = pd.DataFrame(dataset['validation'].select(range(250)))
train_df = train_df[['text', 'summary']]
val_df = train_df[['text', 'summary']]
print(train_df.shape)
print(val_df.shape)
train_df.head()

(2000, 2)
(2000, 2)


,text,summary
0,Сегодня транспортный налог начисляется в завис...,С 2011 года правительство отменяет самый раздр...
1,Словосочетание «музыкальный кинофестиваль» уже...,"Британские затворники, московские модники, бра..."
2,После более чем 12-часовых консультаций Совет ...,Совбез ООН собрался на экстренное совещание дл...
3,"Жертвами урагана «Агата», обрушившегося на Цен...",Ураган «Агата» в Центральной Америке унес жизн...
4,Решение ограничить рост тарифов естественных м...,Правительство хочет сдержать рост тарифов есте...


In [ ]:
class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data.loc[idx]

        messages = [
            {"role": "system", "content": "Ты - AI ассистент для суммаризации новостей на русском языке."},
            {"role": "user", "content": f"Cократи следующий текст с сохранением главной идеи: {example.text}"},
            {"role": "assistant", "content": example.summary}
        ]

        # Применяем чат-темплейт токенизатора
        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )

        # Токенизация
        tokenized = self.tokenizer(
            text,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors=None
        )

        # Для causal LM labels такие же как input_ids
        tokenized["labels"] = tokenized["input_ids"].copy()

        return tokenizer

In [ ]:
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Конфиг квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B-Instruct",
    # quantization_config=bnb_config,
    torch_dtype=torch.float16,
    device_map="auto",
    # trust_remote_code=True,
    use_cache=False  # важно для gradient checkpointing
)

# LoRA конфиг
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    # bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Применяем LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 40,370,176 || all params: 7,655,986,688 || trainable%: 0.5273


In [ ]:
# Создаем датасеты
train_dataset = InstructionDataset(train_df, tokenizer)
val_dataset = InstructionDataset(val_df, tokenizer)

# УНИВЕРСАЛЬНЫЙ КОЛЛАТОР ИЗ HUGGINGFACE
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # важно: False для causal LM
    pad_to_multiple_of=8, # для эффективности на GPU
    return_tensors='pt'
)

# КАСТОМНЫЙ КОЛЛАТОР (запасной вариант)
# def custom_collate_fn(batch):
#     keys = batch[0].keys()
#     collated = {}

#     for key in keys:
#         if key == 'labels':
#             collated[key] = torch.nn.utils.rnn.pad_sequence(
#                 [torch.tensor(item[key]) for item in batch],
#                 batch_first=True,
#                 padding_value=-100
#             )
#         else:
#             collated[key] = torch.nn.utils.rnn.pad_sequence(
#                 [torch.tensor(item[key]) for item in batch],
#                 batch_first=True,
#                 padding_value=tokenizer.pad_token_id
#             )

#     return collated

# DataLoader с универсальным коллатором
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=data_collator  # используем универсальный коллатор
)

val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=data_collator  # используем универсальный коллатор
)

In [ ]:
# Загружаем метрики
rouge = load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # predictions shape: (batch_size, sequence_length, vocab_size)
    predictions = np.argmax(predictions, axis=-1)

    # Декодируем предсказания
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Заменяем -100 в labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Вычисляем ROUGE
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        use_aggregator=True
    )

    # Извлекаем основные метрики ROUGE
    rouge_names = ["rouge1", "rouge2", "rougeL"]
    rouge_dict = {name: round(rouge_result[name].mid.fmeasure, 4) for name in rouge_names}

    return rouge_dict

In [ ]:
# Аргументы обучения
training_args = TrainingArguments(
    output_dir="./qwen_russian_summarization",
    overwrite_output_dir=True,
    num_train_epochs=2,  # Уменьшим для демонстрации
    per_device_train_batch_size=2,  # Уменьшим batch size
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,  # Увеличим accumulation steps
    learning_rate=2e-4,  # Увеличим learning rate для LoRA
    weight_decay=0.01,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    max_grad_norm=0.3,
    gradient_checkpointing=True,

    # Настройки валидации и сохранения
    eval_strategy="epoch",
    # eval_steps=100,
    save_strategy="epoch",
    # save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Настройки логирования
    logging_steps=25,
    logging_dir="./logs",
    report_to=[],

    # Оптимизации
    bf16=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    ddp_find_unused_parameters=False,
)

# Создаем Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()
trainer.save_model("./qwen_russian_summarization_best")

/tmp/ipython-input-3448492372.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


AttributeError: Qwen2TokenizerFast has no attribute size

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./qwen_russian_summarization",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=100,
    lr_scheduler_type="cosine", # linear , constant
    max_grad_norm=1.0,
    # gradient_checkpointing=True,

    # Настройки валидации и сохранения
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Настройки логирования
    logging_steps=50,
    logging_dir="./logs",
    report_to=[],

    # Оптимизации
    # bf16=True,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()
trainer.save_model("./qwen_russian_summarization_best")

/tmp/ipython-input-1123961025.py:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
# Оптимизатор и шедулер
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=2e-5,
    weight_decay=0.01
)

total_steps = len(train_loader) * 3 // 4
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=100,
    num_training_steps=total_steps
)

# Обучение
model.train()
global_step = 0
best_val_loss = float('inf')

for epoch in range(3):
    total_loss = 0
    optimizer.zero_grad()

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for step, batch in enumerate(progress_bar):
        # Перемещаем батч на устройство
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Gradient accumulation
        loss = loss / 4
        loss.backward()

        total_loss += loss.item()

        # Шаг оптимизации
        if (step + 1) % 4 == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1

            # Логирование
            if global_step % 50 == 0:
                current_lr = scheduler.get_last_lr()[0]
                progress_bar.set_postfix({
                    'loss': total_loss / (step + 1),
                    'lr': current_lr,
                    'step': global_step
                })

    # Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for val_batch in val_loader:
            val_batch = {k: v.to(model.device) for k, v in val_batch.items()}
            outputs = model(**val_batch)
            val_loss += outputs.loss.item()

    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}")

    # Сохраняем лучшую модель
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        model.save_pretrained("./best_russian_summarization_model")
        tokenizer.save_pretrained("./best_russian_summarization_model")
        print(f"Новая лучшая модель сохранена с val_loss: {val_loss:.4f}")

    model.train()

Epoch 1:   0%|          | 0/1000 [00:00<?, ?it/s]

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

### causal lm LoRA finetuning (old)

In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

def compute_metrics(preds, targets):
    rouge_results = rouge.compute(
        predictions=preds,
        references=targets,
        use_stemmer=True
    )

    bertscore_results = bertscore.compute(
        predictions=preds,
        references=targets,
        lang="ru"
    )

    return {
        'rouge1': rouge_results['rouge1'],
        'rouge2': rouge_results['rouge2'],
        'rougeL': rouge_results['rougeL'],
        'bertscore_f1': np.mean(bertscore_results['f1'])
    }

In [ ]:
lora_config = LoraConfig(
    task_type='CAUSAL_LM',  # CAUSAL_LM для decoder-only, SEQ_2_SEQ_LM для encoder-decoder
    # inference_mode=False,
    r=8,               # rank матриц LoRA, больше r -> больше параметров, лучше качество но риск переобучения
    lora_alpha=16,      # множитель для масштабирования, обычно r * 4
    lora_dropout=0.1,  # процент дропаута для слоев LoRA для борьбы с переобучением.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],  # модули для qwen, на какие типы весов обучать адаптеры, для разных моделей списки разные!
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,046,272 || all params: 7,620,662,784 || trainable%: 0.0662


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5, weight_decay=0.01)

num_epochs = 3
gradient_accumulation_steps = 4
max_grad_norm = 1.0
best_loss = float('inf')
output_dir = "./qwen25_ckpts"
os.makedirs(output_dir, exist_ok=True)

warmup_ratio = 0.1
total_steps = len(train_loader) * num_epochs // gradient_accumulation_steps
warmup_steps = int(total_steps * warmup_ratio)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

In [ ]:
gc.collect()
torch.cuda.empty_cache()
# del model
# del tokenizer
# del train_dataset
# del val_dataset
# del train_loader
# del val_loader

In [ ]:
for epoch in range(num_epochs):
    # train
    model.train()
    epoch_loss = 0
    accumulation_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for step, batch in enumerate(progress_bar):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        accumulation_loss += loss.item()

        # gradient accumulation step
        if (step + 1) % gradient_accumulation_steps == 0:
            clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            progress_bar.set_postfix({'loss': f'{accumulation_loss:.4f}'})
            accumulation_loss = 0

    # validation
    model.eval()
    val_loss = 0
    val_preds = []
    val_targets = val_df['title'].to_list()
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='validation'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                generation_config=generation_config,
            )
            for idx, val in enumerate(preds):
                pred = val[len(input_ids[idx]):]
                decoded_pred = tokenizer.decode(pred, skip_special_tokens=True)
                val_preds.append(decoded_pred)
    val_metrics = compute_metrics(val_preds, val_targets)
    val_rouge1 = val_metrics['rouge1']
    val_rouge2 = val_metrics['rouge2']
    val_rougel = val_metrics['rougeL']
    val_bertf1 = val_metrics['bertscore_f1']
    val_loss /= len(val_loader)
    print(f"Epoch {epoch+1}: Val Loss = {val_loss:.4f}, ROUGEl = {val_rougel:.4f}, BERT Score F1 = {val_bertf1:.4f}")

    # Сохраняем лучшую модель
    if val_loss < best_loss:
        best_loss = val_loss
        model.save_pretrained(f"{output_dir}/best_model")
        tokenizer.save_pretrained(f"{output_dir}/best_model")
        print(f"New best model: {best_loss:.4f}")

    # Сохраняем чекпоинт эпохи
    model.save_pretrained(f"{output_dir}/epoch-{epoch+1}")
    tokenizer.save_pretrained(f"{output_dir}/epoch-{epoch+1}")

Epoch 1/3:   0%|          | 0/500 [00:00<?, ?it/s]

validation:   0%|          | 0/63 [00:00<?, ?it/s]

Epoch 1: Val Loss = 1.6152, ROUGEl = 0.0046, BERT Score F1 = 0.3710
New best model: 1.6152


Epoch 2/3:   0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# для инференса

# загружаем LoRA адаптеры
model = PeftModel.from_pretrained(model, checkpoint_path)
model.to(device)
model.eval()

# Токенайзер
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

In [ ]:
model_name = "IlyaGusev/saiga_nemo_12b"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# конфигурация 4-битной квантизации
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True, # вложенная квантизация
    bnb_4bit_quant_type="nf4", # тип квантизации (nf4 лучше для обучения)
    bnb_4bit_compute_dtype=torch.float16  # тип для вычислений
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto", # автоматическое распределение по GPU/CPU
    torch_dtype=torch.float16
)
model.to(device)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00001-of-00005.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/273 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(131072, 5120, padding_idx=10)
    (layers): ModuleList(
      (0-39): 40 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=5120, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((5120,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((5120,), eps=1e-05)
   

In [ ]:
print(tokenizer.pad_token)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

<pad>


In [ ]:
generation_config = GenerationConfig.from_pretrained(model_name)
# generation_config.max_new_tokens = 128
# generation_config.length_penalty = 0.7
print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 1024000,
  "max_new_tokens": 2048,
  "pad_token_id": 10,
  "repetition_penalty": 1.1,
  "temperature": 0.6,
  "top_p": 0.95
}



In [ ]:
prompt = tokenizer.apply_chat_template([{
    "role": "user",
    "content": "Сгенерируй заголовок в пару слов для новости: " + val_df.loc[5].summary
}], tokenize=False, add_generation_prompt=True)
data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
data = {k: v.to(model.device) for k, v in data.items()}
data.pop("token_type_ids", None)
output_ids = model.generate(**data, generation_config=generation_config)[0]
output_ids = output_ids[len(data["input_ids"][0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
print(output)

"Трагедия у посольства США в Багдаде"


In [ ]:
class TitlePredictionDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=None, train=False):
        self.df = df
        self.tokenizer = tokenizer
        self.system_prompt = "Ты — AI-ассистент для создания заголовков новостей. \
        # Ты получаешь текст новости и возвращаешь только заголовок (меньше 7 слов) без лишних слов."
        self.base_prompt = "Сгенерируй заголовок в пару слов для новости: "
        self.train = train
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        input = row['summary']

        if self.train:
            target = row['title']

            # для обучения нам нужен полный диалог с ответом ассистента
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.base_prompt + input},
                {"role": "assistant", "content": target}
            ]

            # применяем шаблон для получения отформатированного текста
            full_text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,  # получаем строку, а не токены
                add_generation_prompt=False  # не добавляем промпт генерации, так как он сам добавляется при ответе assistant
            )

            # токенизируем отформатированный текст
            full_text_enc = self.tokenizer(
                full_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            # получим текст БЕЗ ответа ассистента (только промпт)
            prompt_text = self.tokenizer.apply_chat_template(
                messages[:-1],
                tokenize=False,
                add_generation_prompt=True  # добавляем промпт генерации, так как он автоматически не добаляется без ответа assistant
            )

            # токенизируем промпт отдельно
            prompt_enc = self.tokenizer(
                prompt_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt"
            )

            # маскируем промпт, оставляем только ответ
            labels = full_text_enc['input_ids'].squeeze(0).clone()
            prompt_len = prompt_enc['input_ids'].shape[1]
            # маскируем все токены до начала ответа ассистента
            labels[:prompt_len] = -100

            return {
                'input_ids': full_text_enc['input_ids'].squeeze(0),
                'attention_mask': full_text_enc['attention_mask'].squeeze(0),
                'labels': labels
            }

        else:
            messages = [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.base_prompt + input}
            ]

            prompt_text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            prompt_enc = self.tokenizer(
                prompt_text,
                max_length=self.max_len,
                padding=False,
                truncation=True,
                return_tensors="pt",
            )

            return {
                'input_ids': prompt_enc['input_ids'].squeeze(0),
                'attention_mask': prompt_enc['attention_mask'].squeeze(0)
            }

In [ ]:
val_dataset = TitlePredictionDataset(val_df, tokenizer, train=False)

In [ ]:
input_ids = val_dataset[0]['input_ids'].unsqueeze(0).to(device)
att_mask = val_dataset[0]['attention_mask'].unsqueeze(0).to(device)
print(input_ids)
output_ids = model.generate(input_ids=input_ids, attention_mask=att_mask, generation_config=generation_config)[0]
output_ids = output_ids[len(input_ids[0]):]
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
print(output)

tensor([[     1,      1,  11288,   1607,   2251,  26554,  52902,  15733,   1705,
           7902,   7019,  95267,  76639,  36886,   6251,  59039,  56759,   1046,
           1301,   2569,  65790,  43224,  67306,  62951,  59039,   3587,   1756,
          54195,  10996,  67306,  18339,  76639,  36886,   3239,   1319,  85527,
           6349,   1032,   1055,  47197,   1041,  14947,   7010,   2128,  10698,
          47197,   1338,      3,   5111,   1825,  12853,   1964,   2895,   1862,
          76639,  36886,   3239,   1668, 131003,  47197,   7019,  59039,   3587,
           1058,   3192,   2332,  45346,  54495,   9603,   8597,   3420,   3254,
           9612,   1668,  28924,  69798,   1902,  29831, 112433,  35892,   7639,
           4170,   1046,   3192,  94626,  54495,   9603,  78473,   2726,   8349,
           1044,   5858,  18094,  15490,  29988,   7675,  20829,  38018,   1046,
          41550, 107850,  69253,  63015,   2726,  78108,  73320,   1756,  30959,
           1361,   6251,   1

In [ ]:
train_dataset = TitlePredictionDataset(train_df, tokenizer, train=True)
val_dataset = TitlePredictionDataset(val_df, tokenizer, train=False)

In [ ]:
# seq2seq collator паддит и input_ids и labels
train_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, # обязательно tokenizer
    padding='longest', # паддинг до самого длиного элемента в батче
    max_length=None, # так как уже ограничили в датасете
    # pad_to_multiple_of=8, # оптимизация для gpu
)

# обычный collator падит только input_ids?
test_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, # обязательно tokenizer
    padding='longest', # паддинг до самого длиного элемента в батче
    max_length=None, # так как уже ограничили в датасете
    # pad_to_multiple_of=8, # оптимизация для gpu
)

In [ ]:
batch_size=8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=train_collator)

In [ ]:
model.eval()
all_preds = []
# all_targets = []

for batch in tqdm(val_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    # output_ids = model.generate(input_ids=input_ids, attention_mask=att_mask, generation_config=generation_config)[0]
    # output_ids = output_ids[len(input_ids[0]):]
    # output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    # print(output)
    # labels = batch['labels']

    with torch.no_grad():
        preds = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            generation_config=generation_config
        )
    print(preds.shape)
    print(input_ids[0])
    print(attention_mask[0])
    pred = preds[0]
    pred = pred[164:]
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    print(pred)
    # input_lens = attention_mask.sum(dim=1)
    # for idx, val in enumerate(preds):
    #     # input_len = input_lens[idx].item()
    #     # pred = val[input_len:]
    #     decoded_pred = tokenizer.decode(pred, skip_special_tokens=True)
    #     all_preds.append(decoded_pred)
    # print(4, all_preds[0])

    # labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # all_targets.extend(decoded_labels)
    break

  0%|          | 0/38 [00:00<?, ?it/s]

torch.Size([8, 207])
tensor([    10,     10,     10,     10,     10,     10,     10,     10,     10,
            10,     10,     10,     10,     10,     10,     10,     10,     10,
            10,     10,     10,     10,     10,     10,     10,     10,     10,
             1,  11288,   1607,   2251,  26554,  52902,  15733,   1705,   7902,
          7019,  95267,  76639,  36886,   6251,  59039,  56759,   1046,   1301,
          2569,  65790,  43224,  67306,  62951,  59039,   3587,   1756,  54195,
         10996,  67306,  18339,  76639,  36886,   3239,   1319,  85527,   6349,
          1032,   1055,  47197,   1041,  14947,   7010,   2128,  10698,  47197,
          1338,      3,   5111,   1825,  12853,   1964,   2895,   1862,  76639,
         36886,   3239,   1668, 131003,  47197,   7019,  59039,   3587,   1058,
          3192,   2332,  45346,  54495,   9603,   8597,   3420,   3254,   9612,
          1668,  28924,  69798,   1902,  29831, 112433,  35892,   7639,   4170,
          1046,   3

In [ ]:
len([1,      1,  11288,   1607,   2251,  26554,  52902,  15733,   1705,
          7902,   7019,  95267,  76639,  36886,   6251,  59039,  56759,   1046,
          1301,   2569,  65790,  43224,  67306,  62951,  59039,   3587,   1756,
         54195,  10996,  67306,  18339,  76639,  36886,   3239,   1319,  85527,
          6349,   1032,   1055,  47197,   1041,  14947,   7010,   2128,  10698,
         47197,   1338,      3,   5111,   1825,  12853,   1964,   2895,   1862,
         76639,  36886,   3239,   1668, 131003,  47197,   7019,  59039,   3587,
          1058,   3192,   2332,  45346,  54495,   9603,   8597,   3420,   3254,
          9612,   1668,  28924,  69798,   1902,  29831, 112433,  35892,   7639,
          4170,   1046,   3192,  94626,  54495,   9603,  78473,   2726,   8349,
          1044,   5858,  18094,  15490,  29988,   7675,  20829,  38018,   1046,
         41550, 107850,  69253,  63015,   2726,  78108,  73320,   1756,  30959,
          1361,   6251,   1668,   1032,   1050,   1048,   1050,   1048,   9603,
         25659,  97497,   3587,  12828,  11506,  11435,  73194,   8597,   3420,
          3254,   7336,   1046,  27096, 120268,   8349,   2086,  24276,  31005,
         78108,  73320,   1046,  29992,   1699,   1705,   2669,   1668,   5269,
         12853,  27324,   2463,   1044,   7073,  22351,   1481,   1756,   5723,
          2334,   1044,  31607,  59832,  34522,  41277,   3103,  54712,  20832,
          1046,      4])

164

In [ ]:
idx = 5
# print('target:', all_targets[idx])
print('model:', all_preds[idx])

model: ства Украины.Поклонская: Крымчанам отказывают в гражданстве


In [ ]:
rouge = evaluate.load('rouge')
bertscore = evaluate.load('bertscore')

rouge_results = rouge.compute(predictions=all_preds, references=all_targets, use_stemmer=True)
bertscore_results = bertscore.compute(predictions=all_preds, references=all_targets, lang="ru")

for res in rouge_results:
    print(f'{res}: {rouge_results[res]}')
for res in list(bertscore_results.keys())[:-1]:
    print(f'bertscore {res}: {np.mean(bertscore_results[res]).item()}')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

rouge1: 0.03022222222222222
rouge2: 0.0044444444444444444
rougeL: 0.03
rougeLsum: 0.030222222222222223
bertscore precision: 0.6143943337599437
bertscore recall: 0.6634314751625061
bertscore f1: 0.6374868138631185



### causal lm LoRA finetuning (old)

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # CAUSAL_LM для decoder-only, SEQ_2_SEQ_LM для encoder-decoder
    inference_mode=False,
    r=8,               # rank матриц LoRA, больше r -> больше параметров, лучше качество но риск переобучения
    lora_alpha=32,     # множитель для масштабирования, обычно r * 4
    lora_dropout=0.1,  # процент дропаута для слоев LoRA для борьбы с переобучением.
    target_modules=["q_proj", "v_proj"]  # на какие типы весов обучать адаптеры, для разных моделей списки разные!
    # для LLaMA/Mistral: ["q_proj", "k_proj", "v_proj", "o_proj"]
    # для T5: ["q", "v", "k", "o"]
)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,              # загружать модель сразу в 4-битном формате
    bnb_4bit_use_double_quant=True, # включать вложенную квантизацию для большей экономии памяти
    bnb_4bit_quant_type="nf4",      # тип квантизации (nf4 с нормальным распределением или fp4)
    bnb_4bit_compute_dtype=torch.bfloat16 # тип данных для вычислений (обычно bfloat16 для стабильности)
)

In [ ]:
model_name = "ai-forever/ruGPT-3.5-13B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/782 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

pytorch_model-00001-of-00006.bin:   0%|          | 0.00/9.92G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00006.bin:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

ValueError: Target modules {'q_proj', 'v_proj'} not found in the base model. Please check the target modules and try again.

In [ ]:
# 4. Подготовка данных с чат-шаблоном (для decoder-only)
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # Если у токенизатора нет pad_token
def format_function(example):
    # Создаем промпт в формате чата
    prompt = f"<s>[INST] Суммаризуй следующий текст: {example['text']} [/INST]"
    # Объединяем промпт и ответ в одну последовательность для Causal LM
    combined = f"{prompt} {example['summary']} </s>"
    return {"text": combined}

formatted_train_dataset = train_dataset.map(format_function)
formatted_val_dataset = val_dataset.map(format_function)

train_loader = torch.utils.data.DataLoader(
    TextGenerationDataset(formatted_train_dataset, tokenizer, source_key="text", target_key="text"), # source и target теперь одно и то же
    batch_size=2,  # Маленький батч из-за ограничений памяти
    shuffle=True
)

# 5. Цикл обучения вручную (для большего контроля)
optimizer = torch.optim.AdamW(model_for_tuning.parameters(), lr=2e-4)

# Шедулеры
from transformers import get_linear_schedule_with_warmup
num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.1 * num_training_steps, # 10% шагов на "разогрев"
    num_training_steps=num_training_steps
)

# Gradient Clipping
max_grad_norm = 1.0

model_for_tuning.train()
for epoch in range(num_epochs):
    total_loss = 0
    for step, batch in enumerate(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_for_tuning(**batch)
        loss = outputs.loss
        loss.backward()

        # Gradient Clipping
        torch.nn.utils.clip_grad_norm_(model_for_tuning.parameters(), max_grad_norm)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        if step % 100 == 0:
            print(f"Epoch {epoch}, Step {step}, Loss: {loss.item()}")

    # Сохранение чекпоинта после каждой эпохи
    checkpoint_dir = f"./checkpoint-epoch-{epoch}"
    model_for_tuning.save_pretrained(checkpoint_dir)
    print(f"Checkpoint saved to {checkpoint_dir}")

    # Валидация после эпохи
    rouge_val, _, _ = evaluate_model(model_for_tuning, val_loader, tokenizer)
    print(f"Validation ROUGE after epoch {epoch}: {rouge_val}")

# 6. Сохранение и загрузка LoRA весов
model_for_tuning.save_pretrained("./my_lora_model")
# Для загрузки: model = PeftModel.from_pretrained(model, "./my_lora_model")

### vLLM pipeline

In [ ]:
!pip -q install vllm

In [ ]:
import os
from typing import List, Dict

import torch
from vllm import LLM, SamplingParams

In [ ]:
model_name = "Qwen/Qwen2.5-14B-Instruct-AWQ"

llm = LLM(
    model=model_name,
    tokenizer=model_name,

    # параметры загрузки модели
    download_dir="./models",  # директория для кэширования моделей
    load_format="safetensors", # варианты: "auto" (АВТОВЫБОР), "dummy", "pt", "safetensors", "npcache"
    dtype="auto",  # "auto", "half" = "float16", "bfloat16", "float32"

    # параметры памяти
    gpu_memory_utilization=0.7,  # использование памяти GPU (0.0-1.0)
    max_model_len=1024,  # максимальная длина контекста (не ограничена длинной модели)

    # параметры параллелизма
    tensor_parallel_size=1,  # количество GPU для тензорного параллелизма
    pipeline_parallel_size=1,  # размер пайплайна параллелизма (разные слои на разных GPU)

    # параметры quantization (для экономии памяти)
    quantization="awq",  # "awq", "squeezellm", "gptq", None

    # оптимизации
    enable_prefix_caching=True,  # кэширование префиксов промптов для ускорения
    block_size=16,  # размер блока для PagedAttention - если маленький то лучше используется память для коротких запросов,
                    # если большой то меньше расходов для длинных запросов но не оптимально для коротких, средний - 16
    seed=42,
)

# лучше не указывать, определяется по умолчанию
# swap_space=4 - количество доп памяти на cpu в gb, сильно замедляет
# max_num_batched_tokens=8192- максимальный размер батча, собирается на лету
# max_num_seqs=512 - максимальное кол-во последовательностей, обрабатываемых парралельно
# max_logprobs=5 - вернуть топ-5 вероятностей для каждого токена, чисто для анализа

INFO 11-19 21:37:24 [utils.py:253] non-default args: {'tokenizer': 'Qwen/Qwen2.5-14B-Instruct-AWQ', 'download_dir': './models', 'load_format': 'safetensors', 'seed': 42, 'max_model_len': 1024, 'block_size': 16, 'enable_prefix_caching': True, 'gpu_memory_utilization': 0.7, 'disable_log_stats': True, 'quantization': 'awq', 'model': 'Qwen/Qwen2.5-14B-Instruct-AWQ'}
INFO 11-19 21:37:25 [model.py:631] Resolved architecture: Qwen2ForCausalLM
INFO 11-19 21:37:25 [model.py:1745] Using max model len 1024
INFO 11-19 21:37:25 [awq_marlin.py:166] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
INFO 11-19 21:37:25 [scheduler.py:216] Chunked prefill is enabled with max_num_batched_tokens=8192.


Parse safetensors files:   0%|          | 0/3 [00:00<?, ?it/s]

RuntimeError: Engine core initialization failed. See root cause above. Failed core proc(s): {}

Параметры квантизации


1.   AWQ - требует отдельной версии модели, сохраняет наиболее важные веса на основе активаций, ~99% оригинальной точности, оптимизирована для инференса, сокращает память в 3-4 раза
2.   SqueezeLLM - требует отдельной версии модели, адаптивная квантизация с сохранением outlier, лучше сохраняет качество на сложных задачах, сокращает память также в 3-4 раза
3.   GPTQ - post-training quantization с калибровкой, качество немного хуже AWQ, максимальная скорость inference, сокрщает память также примерно в 4 раза
4.   None - без квантизации


In [ ]:
sampling_params = SamplingParams(
    # контроль случайности
    temperature=0.8,
    top_p=0.95,
    top_k=50,

    # длина генерации
    max_tokens=512, # не включая промпт
    min_tokens=10,

    # контроль повторений
    repetition_penalty=1.1,
    frequency_penalty=0.0, # штрафует токены, которые уже встречались много раз (до 1.0, 0.2 - оптимально)
    presence_penalty=0.0, # штрафует токены, которые уже встречались хоть раз (до 1.0, 0.1 - оптимально)

    # остановка
    # stop=["\n", "###"], # стоп-токены
    # stop_token_ids=[2],
    # ignore_eos=False, # False по умолчанию

    # множественная генерация
    n=1, # количество возвращаемых вариантов
    # best_of=3, # количество вариантов для выбора

    # дополнительные
    skip_special_tokens=True, # возвращать без спец-токенов
    spaces_between_special_tokens=True, # пробелы между спец-токенами
)

In [ ]:
# для точных ответов
# sampling_params = SamplingParams(
#     temperature=0.1,           # низкая случайность
#     top_p=0.9,                 # ядерное семплирование
#     top_k=40,                  # ограничение выбора
#     max_tokens=512,
#     repetition_penalty=1.1,    # штраф за повторения
#     frequency_penalty=0.3,     # штраф за частые токены
#     presence_penalty=0.2,      # штраф за упоминания
#     stop=["\n\n", "###"]       # естественные границы
# )

# для креативных задач
# sampling_params = SamplingParams(
#     temperature=0.8,           # высокая случайность
#     top_p=0.95,                # широкий выбор
#     max_tokens=1024,
#     repetition_penalty=1.05,   # легкий штраф
#     frequency_penalty=0.1,     # минимальный штраф
#     presence_penalty=0.1,      # минимальный штраф
#     min_tokens=100             # гарантия развернутого ответа
# )

# для множественных вариантов
# sampling_params = SamplingParams(
#     n=5,                       # 5 вариантов
#     best_of=10,                # выбирать из 10 сгенерированных
#     temperature=0.7,
#     top_p=0.9,
#     max_tokens=256,
#     frequency_penalty=0.2,
#     presence_penalty=0.1
# )

In [ ]:
prompts = [
    "Объясни концепцию машинного обучения простыми словами:",
    "Какие основные типы нейронных сетей существуют?",
    "Опиши процесс обучения модели с учителем:"
]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

def format_chat_prompt(prompt):
    messages = [
        {"role": "system", "content": "Ты полезный AI ассистент."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

In [ ]:
# базовый инференс
outputs = llm.generate([format_chat_prompt(p) for p in prompts], sampling_params, use_tqdm=True)

Adding requests:   0%|          | 0/3 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/3 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [ ]:
outputs[0]

RequestOutput(request_id=0, prompt='<|im_start|>system\nТы полезный AI ассистент.<|im_end|>\n<|im_start|>user\nОбъясни концепцию машинного обучения простыми словами:<|im_end|>\n<|im_start|>assistant\n', prompt_token_ids=[151644, 8948, 198, 33995, 4552, 28519, 31885, 33513, 15235, 20396, 128698, 34011, 18673, 13, 151645, 198, 151644, 872, 198, 73239, 33594, 127023, 22621, 36998, 127759, 133126, 130839, 38800, 143181, 129020, 129129, 91107, 49707, 25, 151645, 198, 151644, 77091, 198], encoder_prompt=None, encoder_prompt_token_ids=None, prompt_logprobs=None, outputs=[CompletionOutput(index=0, text='Машинное обучение - это способность компьютера улучшать свои результаты без явных программирования или указаний человека. Это происходит путем анализа данных и выявления закономерностей в них.\n\nК примеру, предположим, что у вас есть программа для определения спам-писем. Программа может быть обучена на основе большого количества сообщений, которые уже были отмечены как спам или не спам. По мер

In [ ]:
for i, output in enumerate(outputs):
    prompt = output.prompt
    generated_text = output.outputs[0].text

    print(f"Запрос {i+1}:")
    print(f"Промпт: {prompt}")
    print(f"Сгенерированный текст: {generated_text}")
    print(f"Количество токенов: {len(output.outputs[0].token_ids)}")
    print(f"Время генерации: {output.outputs[0].cumulative_logprob}")

Запрос 1:
Промпт: <|im_start|>system
Ты полезный AI ассистент.<|im_end|>
<|im_start|>user
Объясни концепцию машинного обучения простыми словами:<|im_end|>
<|im_start|>assistant

Сгенерированный текст: Машинное обучение - это способность компьютера улучшать свои результаты без явных программирования или указаний человека. Это происходит путем анализа данных и выявления закономерностей в них.

К примеру, предположим, что у вас есть программа для определения спам-писем. Программа может быть обучена на основе большого количества сообщений, которые уже были отмечены как спам или не спам. По мере того, как она анализирует больше и больше писем, она становится всё лучше и лучше в распознавании типов электронной почты, которые обычно считаются спамом.

Суть заключается том, что программа сама по себе "обучается" и адаптируется к новым данным, что позволяет ей становиться всё более точной со временем.
Количество токенов: 195
Время генерации: None
Запрос 2:
Промпт: <|im_start|>system
Ты полезный

### vLLM advanced

In [ ]:
# cложные схемы промптинга и шаблонизация

from vllm import LLM, SamplingParams
import json

llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.1")

def create_competition_prompt(task_type, data, examples=None):
    templates = {
        "classification": """
Анализируй текст и классифицируй его:

Текст: {text}
Категории: {categories}

Примеры:
{examples}

Твой анализ и вывод:
""",
        "extraction": """
Извлеки структурированную информацию из текста:

Текст: {text}
Поля для извлечения: {fields}

Формат ответа: JSON
{examples}

Результат:
"""
    }

    return templates[task_type].format(
        text=data['text'],
        categories=data.get('categories', []),
        fields=data.get('fields', []),
        examples=json.dumps(examples, ensure_ascii=False, indent=2) if examples else "Нет примеров"
    )

# Использование в соревновании
data_batch = [
    {"task": "classification", "data": {"text": "Отличный продукт!", "categories": ["позитив", "негатив", "нейтраль"]}},
    {"task": "extraction", "data": {"text": "Иван купил 5 яблок за 100 рублей", "fields": ["имя", "действие", "количество", "товар", "цена"]}}
]

prompts = []
for item in data_batch:
    prompt = create_competition_prompt(item["task"], item["data"])
    prompts.append(prompt)

sampling_params = SamplingParams(temperature=0.1, max_tokens=200)
results = llm.generate(prompts, sampling_params)

In [ ]:
# Автоматическое CoT для сложных задач:

def chain_of_thought_reasoning(llm, problem, max_steps=3):
    """Многошаговое решение сложных задач"""

    cot_prompt = f"""
Реши задачу шаг за шагом:

Задача: {problem}

Шаг за шагом:
1. Сначала проанализируй:
"""

    reasoning_steps = []
    current_prompt = cot_prompt

    for step in range(max_steps):
        sampling_params = SamplingParams(temperature=0.3, max_tokens=100, stop=["\n\n", "Шаг"])
        output = llm.generate([current_prompt], sampling_params)[0]
        step_reasoning = output.outputs[0].text

        reasoning_steps.append(step_reasoning)

        # Проверяем, пришел ли к заключению
        if "ответ:" in step_reasoning.lower() or "решение:" in step_reasoning.lower():
            break

        # Продолжаем цепочку
        current_prompt += step_reasoning + f"\n{step + 2}. Далее: "

    # Финальный ответ
    final_prompt = current_prompt + "\nИтоговый ответ:"
    final_output = llm.generate([final_prompt], SamplingParams(temperature=0.1, max_tokens=50))
    final_answer = final_output[0].outputs[0].text

    return {
        "reasoning_steps": reasoning_steps,
        "final_answer": final_answer,
        "full_chain": current_prompt + final_answer
    }

# Использование для математических/логических задач
math_problem = "У Маши было 5 яблок. Она отдала 2 яблока Пете, а потом купила еще 4. Сколько яблок у Маши теперь?"
result = chain_of_thought_reasoning(llm, math_problem)

In [ ]:
# Ансамблирование нескольких моделей

class VLLMEnsemble:
    def __init__(self, model_names):
        self.models = {}
        for name in model_names:
            self.models[name] = LLM(model=name)

    def predict_ensemble(self, prompts, weights=None):
        """Предсказание ансамбля моделей"""
        if weights is None:
            weights = {name: 1/len(self.models) for name in self.models}

        all_predictions = {}

        for model_name, model in self.models.items():
            sampling_params = SamplingParams(temperature=0.3, max_tokens=100)
            outputs = model.generate(prompts, sampling_params)
            predictions = [output.outputs[0].text for output in outputs]
            all_predictions[model_name] = predictions

        # Простое усреднение (для численных предсказаний нужно парсить)
        ensemble_results = []
        for i in range(len(prompts)):
            # Собираем все предсказания для данного промпта
            prompt_predictions = []
            for model_name, predictions in all_predictions.items():
                prompt_predictions.extend([predictions[i]] * int(weights[model_name] * 10))

            # Выбираем наиболее частый ответ
            from collections import Counter
            if prompt_predictions:
                final_prediction = Counter(prompt_predictions).most_common(1)[0][0]
            else:
                final_prediction = all_predictions[list(all_predictions.keys())[0]][i]

            ensemble_results.append(final_prediction)

        return ensemble_results, all_predictions

# Использование
ensemble = VLLMEnsemble([
    "mistralai/Mistral-7B-Instruct-v0.1",
    "google/flan-t5-large",  # если поддерживается
    "microsoft/DialoGPT-medium"
])

prompts = ["Что такое машинное обучение?", "Объясни теорию относительности"]
results, individual_preds = ensemble.predict_ensemble(prompts)

In [ ]:
# Исправление ответов

def validate_and_correct(llm, initial_answers, validation_criteria):
    """Проверяет и исправляет ответы модели"""

    correction_prompts = []
    for answer in initial_answers:
        prompt = f"""
Проверь следующий ответ и исправь если нужно:

Исходный ответ: {answer}

Критерии проверки:
{validation_criteria}

Если ответ правильный, повтори его без изменений.
Если есть ошибки, исправь их.

Исправленный ответ:
"""
        correction_prompts.append(prompt)

    sampling_params = SamplingParams(temperature=0.1, max_tokens=150)
    corrected_outputs = llm.generate(correction_prompts, sampling_params)
    corrected_answers = [output.outputs[0].text for output in corrected_outputs]

    return corrected_answers

# Использование
initial_answers = ["15 + 27 = 42", "Столица Франции - Лондон", "Python - компилируемый язык"]
validation_criteria = """
- Математические вычисления должны быть точными
- Географические факты должны быть верными
- Технические утверждения должны быть корректными
"""

corrected = validate_and_correct(llm, initial_answers, validation_criteria)